# Example 2 - Tracking spill

This example defines a digital twin simulation in which several layers (from different materials) of a dike are constructed.

## Necessary Dependencies

In [1]:
# package(s) related to time, space and id
import datetime
import platform

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import shapely.geometry
from simplekml import Kml, Style

# package(s) for data handling
import numpy as np

# digital twin package
import digital_twin.core as core
import digital_twin.model as model

## Create the necessary classes

The Digital Twin core defines several classes which can be combined in mixin classes you can define yourself. For example, in this simulation we define a Site as:
* Identifiable: an object that can be identified with a name 
* Log: an object which will contain a log of events that took place concerning this object after the simulation is complete
* Locatable: an object that has a geographical location
* HasContainer: an object which contains a storage for materials
* HasResource: an object only a limited number of activities can interact with at the same time

In [2]:
Site = type('Site', (core.Identifiable, core.Log, core.Locatable, core.HasContainer, core.HasResource, 
                     core.HasSoil, core.HasSpill, core.HasSpillCondition), {})

Defining mixin classes for our ships and cranes is done similarly. A ship can either be a simple transport barge we will call a TransportResource, or can be a hopper capable of loading and unloading the materials itself (TransportProcessingResource). Finally we define a class for "crane" objects which can be used to load or unload material for a transport barge (ProcessingResource).

In [3]:
TransportResource = type('TransportResource', 
                         (core.Identifiable, core.Log, core.ContainerDependentMovable, 
                          core.HasResource, core.HasPlume, core.HasSoil), {})

TransportProcessingResource = type('TransportProcessingResource', 
                                   (core.Identifiable, core.Log, core.ContainerDependentMovable, 
                                    core.Processor, core.HasResource, core.HasPlume, core.HasSoil), {})
ProcessingResource = type('ProcessingResource', 
                          (core.Identifiable, core.Log, core.Processor, core.HasResource, core.HasPlume), {})

These mixin classes use a couple of core classes not yet introduced:
* Processor: an object which can move materials from one container to another container
* ContainerDependentMovable: this is in fact a combination (extension) of two other core classes:
    * HasContainer: an object which contains a storage for materials
    * Movable: an object which can move from one geographical location to another (extends Locatable, clearly a moving object must keep track of its current location)
  
Defining a mixin class as a ContainerDependentMovable allows for specifying a speed function which depends on the fullness of the storage container. The more basic Movable class uses a fixed speed instead.




## Defining a Simpy Environment

Since the digital twin package uses the Simpy package for its backend, each object we create will need to be passed the same Simpy environment object. This Simpy environment is used during the simulation. In fact, it can be used to start the simulation later on. For now, we need to create the environment object, so we can pass it to each of our mixin objects when we create them. 


In [4]:
my_env = simpy.Environment(initial_time = (datetime.datetime.now() - datetime.datetime(1970, 1, 1)).total_seconds())



## Defining the Sites

For our simulation we need to define Sites for the different material stockpiles, and sites along the length of the dike in which the material needs to be installed. This example includes four material layers: clay, sand, armour and levvel. We need to define a stockpile site for each of these materials. We will store each of the sites defined in this section in a "stocks" list to be used later.


#### Add spill condition

In [5]:
start_1 = datetime.datetime.now()
end_1 = start_1 + datetime.timedelta(seconds = 14*24*3600)

start_2 = datetime.datetime.now() + datetime.timedelta(seconds = 14*24*3600)
end_2 = start_2 + datetime.timedelta(seconds = 28*24*3600)

In [6]:
condition_1 = core.SpillCondition(5_000, start_1, end_1)
condition_2 = core.SpillCondition(5_000, start_2, end_2)

conditions = [condition_1, condition_2]

#### Add soil layers

In [7]:
layer_1 = core.SoilLayer(0, 50_000, "Clay", 1_400, 0.05)
layer_2 = core.SoilLayer(1, 50_000, "Sand", 1_800, 0.3)
layer_3 = core.SoilLayer(2, 50_000, "Rock", 2_000, 0.0)

#### Make (from-) site

In [8]:
data_stock_site = {"env": my_env,
                   "name": "Site_1",
                   "geometry": shapely.geometry.Point(5.0, 52.0),  # lon, lat
                   "capacity": 150000,
                   "level": 150000,
                   "conditions": [condition_1, condition_2]}

# create the site objects
site_1 = Site(**data_stock_site)
site_1.add_layers(soillayers = [layer_2, layer_1, layer_3])

#### Make (to-) site

In [9]:
data_stock_site = {"env": my_env,
                   "name": "Site_2",
                   "geometry": shapely.geometry.Point(6., 53.0),  # lon, lat
                   "capacity": 150000,
                   "level": 0,
                   "conditions": [condition_1, condition_2]}

# create the site objects
site_2 = Site(**data_stock_site)

The constructor parameters needed will differ depending on the core digital twin classes used in the mixin class you've defined. We can best explain each of the parameters we pass in turn:
* __env__: Most core digital twin classes extend a class called SimpyObject and need an "env" construction parameter. These classes, such as HasContainer and HasResource, need you to pass the Simpy environment so they can create Simpy objects within the simulation environment. 
* __name__: An Identifiable object needs a name, which will be used to identify the object.
* __geometry__: A Locatable object needs a geographical location. The object will initially be located at this location. In the case of a Movable object, the geographical location will be updated as it moves. If the object is not movable, the geometry should remain the same throughout the simulation.
* __capacity__: A HasContainer object needs to know the maximum amount of materials it can store.
* __level__: A HasContainer object can optionally be passed the initial level of the container. If no level is passed, the container is initially empty. In the case of our material stockpiles, we want them to be filled instead. 

We could also have passed a __nr_resources__ argument to the constructors to define the number of activities which can use one of these stockpiles at the same time. By default this is set to one, which means only one ship can be loaded at each stockpile at the same time during the simulation.

In addition to the stockpiles, we also need to define the destination sites in which the material will be placed. To do so we will define several sites spaced evenly along the length of the dike for each of the layers. 

## Defining the Equipment

We need to create objects for the equipment used in the simulation. This is done similarly to creating the sites.

In [10]:
# define a simple linear function with which the speed of a ship can be calculated
def compute_v_provider(v_empty, v_full):
    return lambda x: x * (v_full - v_empty) + v_empty

# equipment information in database
data_gantry_crane = {"env": my_env,
                     "name": "Gantry crane",
                     "rate": 0.10}
data_installation_crane = {"env": my_env,
                           "name": "Installation crane",
                           "rate": 0.05}

data_transport_barge_01 = {"env": my_env,
                           "name": "Transport barge 01",
                           "geometry": shapely.geometry.Point(5.019298185633251, 52.94239823421129),  # lon, lat
                           "capacity": 1_000,
                           "compute_v": compute_v_provider(v_empty=1.6, v_full=1)}
data_transport_barge_02 = {"env": my_env,
                           "name": "Transport barge 02",
                           "geometry": shapely.geometry.Point(5.019298185633251, 52.94239823421129),  # lon, lat
                           "capacity": 1_000,
                           "compute_v": compute_v_provider(v_empty=1.6, v_full=1)}

data_hopper = {"env": my_env,
               "name": "Hopper",
               "geometry": shapely.geometry.Point(5.070195628786471, 52.93917167503315),  # lon, lat
               "rate": 2,
               "capacity": 1_000,
               "compute_v": compute_v_provider(v_empty=2, v_full=1.5)}

# create the processing resources
gantry_crane = ProcessingResource(**data_gantry_crane)
installation_crane = ProcessingResource(**data_installation_crane)

# create the transport resources
transport_barge_01 = TransportResource(**data_transport_barge_01)
transport_barge_02 = TransportResource(**data_transport_barge_02)

# create the transport processing resource
hopper = TransportProcessingResource(**data_hopper)

To define our equipment we needed two new constructor arguments:
* __rate__: The speed with which a Processor can process materials
* __compute_v__: A function which given a value in the interval [0,1], representing the fullness of the container, returns the speed of the ContainerDependentMovable

## Defining the Activities

To complete the simulation we need to define which activities need to be completed to complete the simulation. An activity always consists of transporting materials from an origin container to a destination container. We need to define an activity for each origin, destination and the equipment used to transport the materials. For installing the clay layer we will use the transport barges, gantry and installation crane. Since we have two transport barges that could be used, we define two activities for each of the clay layer sites, one for each set of equipment. We also wish to install the clay layer in order, so for each site we need to define a condition that states that the previous site needs to be completed first.

In [11]:
activities = []

activity_1 = model.Activity(env=my_env,
                            name="Soil movement - 1",
                            origin=site_1,
                            destination=site_2,
                            loader=gantry_crane,
                            mover=transport_barge_01,
                            unloader=installation_crane,
                            start_condition=model.TrueCondition())
activity_2 = model.Activity(env=my_env,
                            name="Soil movement - 2",
                            origin=site_1,
                            destination=site_2,
                            loader=gantry_crane,
                            mover=transport_barge_02,
                            unloader=installation_crane,
                            start_condition=model.TrueCondition())

activities.append(activity_1)
activities.append(activity_2)

## Running the Simulation

To run the simulation, we now simply need to tell the Simpy environment to run. This will run all activities (most of which will first suspend themselved unstill the start_condition is satisfied) and terminate once every Activity has reached its stop_condition. To allow for the visualisation later, we record the start time in the environment.

In [12]:
#my_env.epoch = datetime.datetime.now()
my_env.run()

print("\n*** Installation of dike finished in {} ***".format(datetime.timedelta(seconds=int(my_env.now))))

T=1547216037.15 Start condition is satisfied, Soil movement - 1 transporting from Site_1 to Site_2 started
Using Transport barge 01 to process 1000
T=1547216037.15 Start condition is satisfied, Soil movement - 2 transporting from Site_1 to Site_2 started
Using Transport barge 02 to process 1000
Moved:
  object:      Transport barge 01 contains: 0
  from:        5.01930 52.94240
  to:          5.00000 52.00000
Moved:
  object:      Transport barge 02 contains: 0
  from:        5.01930 52.94240
  to:          5.00000 52.00000
50000
Processed 1000:
  from:        Site_1 contains: 149000
  by:          Gantry crane
  to:          Transport barge 01 contains: 1000
49000
Processed 1000:
  from:        Site_1 contains: 148000
  by:          Gantry crane
  to:          Transport barge 02 contains: 1000
Moved:
  object:      Transport barge 01 contains: 1000
  from:        5.00000 52.00000
  to:          6.00000 53.00000
Moved:
  object:      Transport barge 02 contains: 1000
  from:        5.0

Moved:
  object:      Transport barge 02 contains: 1000
  from:        5.00000 52.00000
  to:          6.00000 53.00000
Processed 1000:
  from:        Transport barge 01 contains: 0
  by:          Installation crane
  to:          Site_2 contains: 51000
Using Transport barge 01 to process 1000
Processed 1000:
  from:        Transport barge 02 contains: 0
  by:          Installation crane
  to:          Site_2 contains: 52000
Using Transport barge 02 to process 1000
Moved:
  object:      Transport barge 01 contains: 0
  from:        6.00000 53.00000
  to:          5.00000 52.00000
0
48000
Processed 1000:
  from:        Site_1 contains: 97000
  by:          Gantry crane
  to:          Transport barge 01 contains: 1000
Moved:
  object:      Transport barge 02 contains: 0
  from:        6.00000 53.00000
  to:          5.00000 52.00000
0
47000
Processed 1000:
  from:        Site_1 contains: 96000
  by:          Gantry crane
  to:          Transport barge 02 contains: 1000
Moved:
  object:  

  from:        6.00000 53.00000
  to:          5.00000 52.00000
0
13000
Processed 1000:
  from:        Site_1 contains: 62000
  by:          Gantry crane
  to:          Transport barge 02 contains: 1000
Moved:
  object:      Transport barge 01 contains: 1000
  from:        5.00000 52.00000
  to:          6.00000 53.00000
Moved:
  object:      Transport barge 02 contains: 1000
  from:        5.00000 52.00000
  to:          6.00000 53.00000
Processed 1000:
  from:        Transport barge 01 contains: 0
  by:          Installation crane
  to:          Site_2 contains: 87000
Using Transport barge 01 to process 1000
Processed 1000:
  from:        Transport barge 02 contains: 0
  by:          Installation crane
  to:          Site_2 contains: 88000
Using Transport barge 02 to process 1000
Moved:
  object:      Transport barge 01 contains: 0
  from:        6.00000 53.00000
  to:          5.00000 52.00000
0
12000
Processed 1000:
  from:        Site_1 contains: 61000
  by:          Gantry crane


Moved:
  object:      Transport barge 01 contains: 0
  from:        6.00000 53.00000
  to:          5.00000 52.00000
0
0
30000
Processed 1000:
  from:        Site_1 contains: 29000
  by:          Gantry crane
  to:          Transport barge 01 contains: 1000
Moved:
  object:      Transport barge 02 contains: 0
  from:        6.00000 53.00000
  to:          5.00000 52.00000
0
0
29000
Processed 1000:
  from:        Site_1 contains: 28000
  by:          Gantry crane
  to:          Transport barge 02 contains: 1000
Moved:
  object:      Transport barge 01 contains: 1000
  from:        5.00000 52.00000
  to:          6.00000 53.00000
Moved:
  object:      Transport barge 02 contains: 1000
  from:        5.00000 52.00000
  to:          6.00000 53.00000
Processed 1000:
  from:        Transport barge 01 contains: 0
  by:          Installation crane
  to:          Site_2 contains: 121000
Using Transport barge 01 to process 1000
Processed 1000:
  from:        Transport barge 02 contains: 0
  by: 

## Analysing the Results

In addition to the print statements provided by the Activity class, it also enters log entries into the logs of the origin, destination, mover, loader and unloader to mark when events such as loading or unloading the mover took place. For the mover, origin and destination, these log entries also contain the level of the container at the time of the log entry. This can be used for some basic visualisation of the simulation in Google Earth. 

Using the information in the log you could also create plots of the usage of each piece of equipment during the simulation. For example, the code below creates a plot of the usage of the two transporting barges.

In [13]:
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
class plot:
    """plot class"""

    def vessel_planning(vessels, activities, colors, web=False):
        """create a plot of the planning of vessels"""

        def get_segments(series, activity, y_val):
            """extract 'start' and 'stop' of activities from log"""
            x = []
            y = []
            for i, v in series.iteritems():
                if v == activity + ' start':
                    start = i
                if v == activity + ' stop':
                    x.extend((start, start, i, i, i))
                    y.extend((y_val, y_val, y_val, y_val, None))
            return x, y

        # organise logdata into 'dataframes' 
        dataframes = []
        for vessel in vessels:
            df = pd.DataFrame(
                {'log_value': vessel.value, 'log_string': vessel.log}, vessel.t)
            dataframes.append(df)
        df = dataframes[0]

        # prepare traces for each of the activities
        traces = []
        for i, activity in enumerate(activities):
            x_combined = []
            y_combined = []
            for k, df in enumerate(dataframes):
                y_val = vessels[k].name
                x, y = get_segments(
                    df['log_string'], activity=activity, y_val=y_val)
                x_combined.extend(x)
                y_combined.extend(y)
            traces.append(go.Scatter(
                name=activity,
                x=x_combined,
                y=y_combined,
                mode='lines',
                hoverinfo='y+name',
                line=dict(color=colors[i], width=10),
                connectgaps=False))
        
        # prepare layout of figure
        layout = go.Layout(
            title='Vessel planning',
            hovermode='closest',
            legend=dict(x=0, y=-.2, orientation="h"),
            xaxis=dict(
                title='Time',
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f'),
                range=[0, vessel.t[-1]]),
            yaxis=dict(
                title='Vessels',
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f')))
        
        # plot figure
        init_notebook_mode(connected=True)        
        fig = go.Figure(data=traces, layout=layout)
        return iplot(fig, filename='news-source')

In [14]:
vessels = [transport_barge_01, transport_barge_02]
activities = ['loading', 'unloading', 'sailing full', 'sailing empty', 'waiting for spill']
colors = {0:'rgb(55,126,184)', 1:'rgb(77,175,74)', 2:'rgb(255,0,0)', 3:'rgb(255,150,0)', 4:'rgb(255, 0, 0)'}

plot.vessel_planning(vessels, activities, colors)

### Check sites

In [15]:
site_1.soil

{'Layer 0000': {'Layer': 0,
  'Volume': 0,
  'Material': 'Clay',
  'Density': 1400,
  'Fines': 0.05},
 'Layer 0001': {'Layer': 1,
  'Volume': 0,
  'Material': 'Sand',
  'Density': 1800,
  'Fines': 0.3},
 'Layer 0002': {'Layer': 2,
  'Volume': 0,
  'Material': 'Rock',
  'Density': 2000,
  'Fines': 0.0}}

In [16]:
site_2.soil

{'Layer 0000': {'Layer': 0,
  'Volume': 50000,
  'Material': 'Mixture of: Rock',
  'Density': 2000.0,
  'Fines': 0.0},
 'Layer 0001': {'Layer': 1,
  'Volume': 50000,
  'Material': 'Mixture of: Sand',
  'Density': 1800.0,
  'Fines': 0.3},
 'Layer 0002': {'Layer': 2,
  'Volume': 50000,
  'Material': 'Mixture of: Clay',
  'Density': 1400.0,
  'Fines': 0.05}}